# 03 — Line sensors + Line following (TurboPi)

This notebook has two parts:

## 03a) Sensor sanity
- Prints live infrared (line) sensor readings so you can confirm wiring + expected values.

## 03b) Line follow
- Uses the IR sensors to follow a line (straight + curves).
- Includes safe stop + timeout.

✅ Uses `fast_sdk.infra_red.InfraredSensors` and `fast_sdk.motors.ControlChassis`.

**Safety:** keep the robot on a stand / wheels off the ground for the first run.


In [ ]:
# --- Optional: show runtime details ---
import sys, os
print('Python:', sys.version.split()[0])
print('CWD:', os.getcwd())


In [ ]:
# --- Imports (the ones that worked on your robot image) ---
import time
import inspect

from fast_sdk.infra_red import InfraredSensors
from fast_sdk.motors import ControlChassis

ir = InfraredSensors()
ch = ControlChassis()

print('InfraredSensors:', ir)
print('ControlChassis:', ch)

# Discover available methods (helps when SDKs differ)
print('\nIR candidates:', [n for n in dir(ir) if any(k in n.lower() for k in ('read', 'sensor', 'infra'))])
print('Chassis candidates:', [n for n in dir(ch) if any(k in n.lower() for k in ('set', 'vel', 'move', 'stop', 'reset', 'translation'))])


## Helpers

These helpers:
- read IR values from whatever method exists on your image
- normalise to a list of ints
- drive the chassis in a consistent way


In [ ]:
def _try_call(obj, method_names):
    """Return (name, value) for the first method that works, else (None, None)."""
    for name in method_names:
        if not hasattr(obj, name):
            continue
        fn = getattr(obj, name)
        if not callable(fn):
            continue
        try:
            return name, fn()
        except TypeError:
            # Some APIs might need args; skip
            continue
        except Exception:
            continue
    return None, None


def read_ir_values():
    """Best-effort read of line sensor values from InfraredSensors."""
    # Common method names seen across images
    candidates = [
        'read_sensor_data',
        'read',
        'get_sensor_data',
        'get_data',
        'read_data',
        'read_infrared',
    ]
    name, raw = _try_call(ir, candidates)
    if name is None:
        raise RuntimeError('Could not find a working IR read method on InfraredSensors')

    # Normalise raw -> list[int]
    if raw is None:
        raise RuntimeError(f'{name}() returned None')

    if isinstance(raw, (list, tuple)):
        vals = list(raw)
    elif isinstance(raw, dict):
        # If dict, try stable ordering
        vals = [raw[k] for k in sorted(raw.keys())]
    else:
        # single value
        vals = [raw]

    # Cast to ints where possible
    out = []
    for v in vals:
        try:
            out.append(int(v))
        except Exception:
            out.append(v)

    return name, out


def chassis_stop():
    # Different SDKs use different stop methods
    for name in ('reset_motors', 'stop', 'stop_all', 'brake'):
        if hasattr(ch, name):
            try:
                getattr(ch, name)()
                return
            except Exception:
                pass

    # Fallback: try set_velocity zeros
    if hasattr(ch, 'set_velocity'):
        try:
            ch.set_velocity(0, 0, 0)
            return
        except Exception:
            pass


def chassis_drive(speed: float, turn: float):
    """Drive forward with a turn rate. Keeps API flexible."""
    # Common pattern in your earlier movement demo: set_velocity(speed, direction_deg, yaw_rate)
    if hasattr(ch, 'set_velocity'):
        try:
            # Forward direction=0, turn positive/negative
            ch.set_velocity(float(speed), 0, float(turn))
            return
        except Exception as e:
            raise RuntimeError(f'ch.set_velocity failed: {e}')

    # Fallback to translation if present
    if hasattr(ch, 'translation'):
        try:
            ch.translation(float(speed), float(turn))
            return
        except Exception as e:
            raise RuntimeError(f'ch.translation failed: {e}')

    raise RuntimeError('No known drive method found on ControlChassis')


## 03a — Sensor sanity (live readings)

Run this with the robot over:
- plain floor
- the line

You’re looking for a clear difference between sensors.


In [ ]:
SAMPLE_SECONDS = 6.0
INTERVAL_S = 0.15

end = time.time() + SAMPLE_SECONDS
print('Reading IR sensors for', SAMPLE_SECONDS, 'seconds...')

first_method = None
count = 0
while time.time() < end:
    method, vals = read_ir_values()
    if first_method is None:
        first_method = method
        print('Using IR read method:', first_method)
    print(f'{count:03d}  IR: {vals}')
    count += 1
    time.sleep(INTERVAL_S)

print('Done.')


## 03b — Line follow (straight + curves)

### How it works
- We read N sensors (3 or 5 are most common)
- Convert sensor readings to a left/right **error**
- Turn proportional to that error (plus a bit of D-term)

### Tuning knobs
- `BASE_SPEED` (go slower for tight curves)
- `KP`, `KD`
- `LINE_IS_LOW` (if your sensor reports **0 on line**)

✅ Use `Ctrl+C` or stop the cell to halt.


In [ ]:
# ---- Tune these ----
BASE_SPEED = 55        # forward speed
KP = 28.0              # proportional gain
KD = 10.0              # derivative gain

# Many IR line sensors report 0 when on black line and 1 on white floor.
# If your robot turns the wrong way or never detects the line, flip this.
LINE_IS_LOW = True

LOOP_HZ = 25
RUN_SECONDS = 12.0

# Safety: if line is lost for too long, stop.
MAX_LOST_COUNT = int(0.8 * LOOP_HZ)  # ~0.8 seconds


def weights_for(n):
    if n == 3:
        return [-1.0, 0.0, 1.0]
    if n == 5:
        return [-2.0, -1.0, 0.0, 1.0, 2.0]
    # Generic symmetric weights
    mid = (n - 1) / 2.0
    return [i - mid for i in range(n)]


def to_line_strength(vals):
    """Convert raw sensor vals into 'line strength' per sensor in [0..1]."""
    # If values are 0/1, treat as digital
    # If values are larger (analogue), normalise by max
    v = [float(x) for x in vals]
    mx = max(v) if len(v) else 1.0
    mn = min(v) if len(v) else 0.0

    # Detect digital-ish
    unique = set(int(round(x)) for x in v)
    is_digital = unique.issubset({0, 1})

    if is_digital:
        # line is either low(0) or high(1)
        if LINE_IS_LOW:
            # on-line => 0 becomes 1.0 strength
            return [1.0 - float(int(round(x))) for x in v]
        else:
            # on-line => 1 becomes 1.0 strength
            return [float(int(round(x))) for x in v]

    # Analogue-ish
    # Assume line is low unless configured otherwise
    if abs(mx - mn) < 1e-6:
        return [0.0 for _ in v]

    if LINE_IS_LOW:
        # low value means stronger line
        return [(mx - x) / (mx - mn) for x in v]
    else:
        return [(x - mn) / (mx - mn) for x in v]


def compute_error(vals):
    """Return (error, strength_list, total_strength)."""
    strengths = to_line_strength(vals)
    w = weights_for(len(strengths))
    total = sum(strengths)
    if total <= 1e-6:
        return 0.0, strengths, 0.0
    pos = sum(si * wi for si, wi in zip(strengths, w)) / total
    # error: negative means line left, positive means line right
    return float(pos), strengths, float(total)


print('Starting line follow...')
print('BASE_SPEED=', BASE_SPEED, 'KP=', KP, 'KD=', KD, 'LINE_IS_LOW=', LINE_IS_LOW)

dt = 1.0 / LOOP_HZ
end = time.time() + RUN_SECONDS
prev_err = 0.0
lost = 0

try:
    while time.time() < end:
        _, raw = read_ir_values()
        err, strengths, total = compute_error(raw)

        # If we lost the line, try a small search turn
        if total <= 1e-6:
            lost += 1
            turn = 35.0 if prev_err >= 0 else -35.0
            chassis_drive(0.0, turn)
            if lost >= MAX_LOST_COUNT:
                print('Line lost too long -> stopping')
                break
            time.sleep(dt)
            continue
        else:
            lost = 0

        derr = (err - prev_err) / dt
        prev_err = err

        turn = KP * err + KD * derr
        # clamp turn to something sane
        if turn > 80:
            turn = 80
        if turn < -80:
            turn = -80

        # Slow down slightly for tight turns
        speed = float(BASE_SPEED)
        if abs(err) > 1.2:
            speed *= 0.75

        chassis_drive(speed, -turn)  # sign: adjust if it steers opposite

        print(f'IR={raw}  strength={list(map(lambda x: round(x,2), strengths))}  err={err:+.2f}  turn={turn:+.1f}  speed={speed:.1f}')
        time.sleep(dt)

finally:
    chassis_stop()
    print('Stopped.')


## If it turns the wrong way

Two common fixes:
1) Flip steering sign: change `chassis_drive(speed, -turn)` to `chassis_drive(speed, +turn)`
2) Flip line polarity: set `LINE_IS_LOW = False`

If you paste 5–10 lines of the `IR=... strength=... err=...` output, I can tell you which one it is.
